In [5]:
import os
from pathlib import Path
import pandas as pd
from pandas_datareader import data as pdr
import numpy as np
import math
import datetime as dt
import matplotlib.pyplot as plt
import yfinance as yf

# plotting configuration
plt.rcParams.update({
    'figure.figsize': (11,6),
    'figure.dpi': 120,
    'axes.grid': True,
    'grid.alpha': 0.25,
})


# make output directories
OUT_DIR = Path("outputs")
FIG_DIR = OUT_DIR / "figs"
OUT_DIR.mkdir(exist_ok=True)
FIG_DIR.mkdir(exist_ok=True)

In [6]:
# TICKER mapping: 
TICKERS = {
    'SPY': 'SPY',
    'QQQ': 'QQQ',
    'VXUS': 'VXUS',
    'BND': 'BND',
    'IEF': 'IEF',
    'TIP': 'TIP', # Yahoo ticker for TIPS ETF is TIP
    'SHY': 'SHY',
    'GLD': 'GLD',
    # Crypto
    'BTC': 'BTC-USD',
    'ETH': 'ETH-USD',
    'SOL': 'SOL-USD'
}


START_DATE = '1995-01-01' # covers optional 1997 event
END_DATE = dt.date.today().isoformat()


# portfolio sleeve targets
TOTAL_TARGET = {'equity': 0.60, 'bond': 0.30, 'cash': 0.10, 'gold': 0.0}
EQUITY_BREAKDOWN = {'SPY': 0.7, 'QQQ': 0.2, 'VXUS': 0.1}
BOND_BREAKDOWN = {'BND': 0.6667, 'IEF': 0.16665, 'TIP': 0.16665}
CASH_ASSET = 'SHY'
GOLD_ASSET = 'GLD'


# whether to include crypto variant
INCLUDE_CRYPTO_VARIANT = False


# Event windows 
EVENTS = {
    "Dotcom": ("2000-03-01", "2002-10-31"),
    "GFC": ("2007-10-01", "2009-03-31"),
    "COVID": ("2020-02-01", "2020-04-30"),
    "2022_rates": ("2022-01-01", "2022-10-31"),
    
    "Asian_1997": ("1997-07-01", "1998-12-31"),
    "Debt_Ceiling_2011": ("2011-08-01", "2011-12-31"),
}


# --------------------
# small helper flags
VERBOSE = True

## Helper functions: download data, simulate monthly rebalancing, compute metrics, plotting helpers.

In [7]:
def download_prices(tickers_map, start, end):
    """
    Download Close prices (price-only) from Yahoo for each symbol in tickers_map.
    Returns a DataFrame with friendly names as columns and a DateTime index.
    """
    yahoo_symbols = list(tickers_map.values())
    print("Downloading:", ", ".join(yahoo_symbols))
    raw = yf.download(yahoo_symbols, start=start, end=end, progress=False)
    
    
    # raw may be a DataFrame with multi-level columns when multiple fields requested; we ask for 'Close'
    if isinstance(raw, pd.DataFrame) and 'Close' in raw.columns:
        df = raw['Close'].copy()
    else:
        # sometimes yf.download returns a DataFrame of closes directly or a Series for single symbol
        df = raw.copy()
    
    
    # convert Series -> DataFrame if necessary
    if isinstance(df, pd.Series):
        df = df.to_frame()
    
    
    # rename columns using reverse mapping
    revmap = {v: k for k, v in tickers_map.items()}
    # if df columns are yahoo tickers, map them; otherwise keep as-is
    df = df.rename(columns=lambda c: revmap.get(c, c))
    
    
    df.index = pd.to_datetime(df.index)
    df = df.sort_index().ffill().dropna(how='all')
    return df


# Download prices once 
prices = download_prices(TICKERS, START_DATE, END_DATE)
print("Downloaded columns:", list(prices.columns))
# quick preview
display(prices.iloc[-5:])

Downloading: SPY, QQQ, VXUS, BND, IEF, TIP, SHY, GLD, BTC-USD, ETH-USD, SOL-USD


C:\Users\user\AppData\Local\Temp\ipykernel_8044\1715454124.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  raw = yf.download(yahoo_symbols, start=start, end=end, progress=False)


Downloaded columns: ['BND', 'BTC', 'ETH', 'GLD', 'IEF', 'QQQ', 'SHY', 'SOL', 'SPY', 'TIP', 'VXUS']


Ticker,BND,BTC,ETH,GLD,IEF,QQQ,SHY,SOL,SPY,TIP,VXUS
Date,,,,,,,,,,,
2025-08-30,73.557007,108808.070312,4374.153320,318.070007,95.835999,570.400024,82.722008,202.860138,645.049988,110.752998,71.370003
2025-08-31,73.557007,108236.710938,4390.019043,318.070007,95.835999,570.400024,82.722008,200.863541,645.049988,110.752998,71.370003
2025-09-01,73.557007,109250.593750,4314.470215,318.070007,95.835999,570.400024,82.722008,197.108337,645.049988,110.752998,71.370003
2025-09-02,73.400002,111200.585938,4325.365723,325.589996,95.550003,565.619995,82.680000,209.481537,640.270020,110.550003,70.870003
2025-09-03,73.639999,111723.210938,4450.389160,328.140015,95.879997,570.070007,82.739998,210.746307,643.739990,110.800003,71.019997


## Compute daily returns from Price series. We use price-only returns.


In [8]:
returns = prices.pct_change()
returns = returns.dropna(how='all')


# preview
returns.iloc[-5:].round(6)

Ticker,BND,BTC,ETH,GLD,IEF,QQQ,SHY,SOL,SPY,TIP,VXUS
Date,,,,,,,,,,,
2025-08-30,0.000000,0.003664,0.003211,0.000000,0.000000,0.000000,0.000000,-0.011499,0.00000,0.000000,0.000000
2025-08-31,0.000000,-0.005251,0.003627,0.000000,0.000000,0.000000,0.000000,-0.009842,0.00000,0.000000,0.000000
2025-09-01,0.000000,0.009367,-0.017209,0.000000,0.000000,0.000000,0.000000,-0.018695,0.00000,0.000000,0.000000
2025-09-02,-0.002134,0.017849,0.002525,0.023643,-0.002984,-0.008380,-0.000508,0.062774,-0.00741,-0.001833,-0.007006
2025-09-03,0.003270,0.004700,0.028905,0.007832,0.003454,0.007867,0.000726,0.006038,0.00542,0.002261,0.002116


## Build portfolio target weights from the CONFIG. This creates per-asset weights so that the balanced portfolio is 60% equity, 30% bond, 10% cash (with equity/bond internal splits).

In [9]:
def build_weights(total_target, equity_break, bond_break, cash_asset, gold_asset):
    w = {}
    # equities
    for t, p in equity_break.items():
        w[t] = total_target['equity'] * p
    # bonds
    for t, p in bond_break.items():
        w[t] = total_target['bond'] * p
    # cash
    w[cash_asset] = total_target['cash']
    # gold
    w[gold_asset] = total_target.get('gold', 0.0)
    return w


weights = build_weights(TOTAL_TARGET, EQUITY_BREAKDOWN, BOND_BREAKDOWN, CASH_ASSET, GOLD_ASSET)
# keep only tickers present in prices
weights = {k: v for k, v in weights.items() if k in prices.columns}


print("Using weights for the following assets:")
for k,v in weights.items():
    print(f" {k}: {v:.4f}")

Using weights for the following assets:
 SPY: 0.4200
 QQQ: 0.1200
 VXUS: 0.0600
 BND: 0.2000
 IEF: 0.0500
 TIP: 0.0500
 SHY: 0.1000
 GLD: 0.0000


## Simulate monthly rebalancing using a shares-based approach. Start with portfolio value = 100. Monthly rebalances are executed on month-end dates (pandas .is_month_end property).

In [10]:
def simulate_monthly_rebalance(prices_df, weights, start_value=100.0):
    assets = list(weights.keys())
    p = prices_df[assets].ffill().dropna()
    
    
    dates = p.index
    holdings = pd.DataFrame(index=dates, columns=assets, dtype=float)
    port_value = pd.Series(index=dates, dtype=float)
    
    
    # initial allocation at first available date
    first_date = dates[0]
    first_prices = p.loc[first_date]
    shares = {a: (start_value * weights[a]) / float(first_prices[a]) for a in assets}
    holdings.loc[first_date] = pd.Series(shares)
    port_value.loc[first_date] = (holdings.loc[first_date] * first_prices).sum()
    
    
    # iterate safely using index positions
    for i in range(1, len(dates)):
        date = dates[i]
        prev_date = dates[i-1]
        prev_shares = holdings.loc[prev_date]
        today_prices = p.loc[date]
        # update portfolio value based on previous shares
        port_value.loc[date] = (prev_shares * today_prices).sum()
        # carry forward holdings
        holdings.loc[date] = prev_shares
        # rebalance on month-end
        if date.is_month_end:
            total = port_value.loc[date]
            new_shares = {a: (total * weights[a]) / float(today_prices[a]) for a in assets}
            holdings.loc[date] = pd.Series(new_shares)
            port_value.loc[date] = (holdings.loc[date] * today_prices).sum()
    
    
    holdings = holdings.ffill()
    port_vals_check = (holdings * p).sum(axis=1)
    port_vals_check = port_vals_check.ffill()
    
    
    return port_vals_check, holdings

# run simulation
port_vals, holdings = simulate_monthly_rebalance(prices, weights, start_value=100.0)


# quick check
print("Portfolio sample values:")
print(port_vals.iloc[-5:])


# save time series for later use
port_vals.to_csv(OUT_DIR / 'balanced_timeseries.csv')


# benchmark SPY series
if 'SPY' in prices.columns:
    spy_series = prices['SPY'].ffill().dropna()
    spy_series.to_csv(OUT_DIR / 'spy_timeseries.csv')
else:
    spy_series = None
    print('Warning: SPY not present in downloaded prices')

Portfolio sample values:
Date
2025-08-30    365.350638
2025-08-31    365.350638
2025-09-01    365.350638
2025-09-02    363.430069
2025-09-03    365.012673
dtype: float64


<h2>Metrics helpers. For a given price series within an event window compute:
- CAGR (annualized)
- Annualized volatility
- Max drawdown
- Peak-to-trough crash return
- Recovery time in days (from trough back to pre-crash peak)</h2>

In [11]:
def compute_event_metrics(price_series):
    price_series = price_series.dropna()
    if len(price_series) < 2:
        return None
    # returns
    returns = price_series.pct_change().dropna()
    # cumulative normalized (start = 1)
    cum = price_series / price_series.iloc[0]
    
    
    vol = returns.std() * math.sqrt(252)
    running_max = cum.cummax()
    drawdown = cum / running_max - 1.0
    max_dd = drawdown.min()
    trough_idx = drawdown.idxmin()
    trough_val = cum.loc[trough_idx]
    peak_before_trough = cum.loc[:trough_idx].max()
    crash_return = trough_val / peak_before_trough - 1.0 if peak_before_trough > 0 else np.nan
    post_trough = cum.loc[trough_idx:]
    recovered = post_trough[post_trough >= peak_before_trough]
    recovery_days = (recovered.index[0] - trough_idx).days if len(recovered) > 0 else np.nan
    
    
    years = (price_series.index[-1] - price_series.index[0]).days / 365.25
    cagr = (price_series.iloc[-1] / price_series.iloc[0]) ** (1/years) - 1 if years > 0 else np.nan
    
    
    return {
        'CAGR': cagr,
        'Volatility': vol,
        'MaxDrawdown': max_dd,
        'CrashReturn': crash_return,
        'RecoveryDays': recovery_days,
        'TroughDate': trough_idx.date(),
        'StartDate': price_series.index[0].date(),
        'EndDate': price_series.index[-1].date()
    }


# quick test on full series
if len(port_vals) > 0:
    test_metrics = compute_event_metrics(port_vals)
    print('Full-range check metrics sample:', test_metrics)

Full-range check metrics sample: {'CAGR': np.float64(0.09274527615282957), 'Volatility': 0.09073538270882531, 'MaxDrawdown': -0.2129707418980259, 'CrashReturn': np.float64(-0.2129707418980259), 'RecoveryDays': 467, 'TroughDate': datetime.date(2022, 10, 14), 'StartDate': datetime.date(2011, 1, 28), 'EndDate': datetime.date(2025, 9, 3)}


## Per-event analysis loop: slice the port and spy series for each event, compute metrics, make plots, and save outputs.

In [13]:
rows = []

# Helpful: print data coverage for key series
print("Balanced portfolio available from:", port_vals.index.min().date(), "to", port_vals.index.max().date())
if spy_series is not None:
    print("SPY available from:", spy_series.index.min().date(), "to", spy_series.index.max().date())

for ev_name, (s, e) in EVENTS.items():
    s_dt = pd.to_datetime(s)
    e_dt = pd.to_datetime(e)

    # Slice by date range independently for each series to avoid boolean mask length issues
    port_slice = port_vals.loc[s_dt:e_dt].dropna()
    spy_slice = spy_series.loc[s_dt:e_dt].dropna() if spy_series is not None else None

    # Require at least 30 trading days for a meaningful window
    if len(port_slice) < 30:
        if VERBOSE:
            print(f"Skipping {ev_name}: insufficient Balanced data in {s}→{e} (have {len(port_slice)} days)")
        continue
    if spy_slice is not None and len(spy_slice) < 30:
        if VERBOSE:
            print(f"Skipping {ev_name}: insufficient SPY data in {s}→{e} (have {len(spy_slice)} days)")
        spy_slice = None  # proceed with just Balanced if you want

    # normalize to start=100 for plotting
    port_norm = port_slice / port_slice.iloc[0] * 100.0
    spy_norm = spy_slice / spy_slice.iloc[0] * 100.0 if spy_slice is not None else None

    # compute metrics
    m_port = compute_event_metrics(port_slice)
    m_spy = compute_event_metrics(spy_slice) if spy_slice is not None else None

    rows.append({
        'Event': ev_name,
        'Ticker': 'Balanced_60_30_10',
        'CAGR': m_port['CAGR'],
        'Volatility': m_port['Volatility'],
        'MaxDrawdown': m_port['MaxDrawdown'],
        'CrashReturn': m_port['CrashReturn'],
        'RecoveryDays': m_port['RecoveryDays'],
        'StartDate': m_port['StartDate'],
        'EndDate': m_port['EndDate'],
        'TroughDate': m_port['TroughDate']
    })
    if m_spy is not None:
        rows.append({
            'Event': ev_name,
            'Ticker': 'SPY',
            'CAGR': m_spy['CAGR'],
            'Volatility': m_spy['Volatility'],
            'MaxDrawdown': m_spy['MaxDrawdown'],
            'CrashReturn': m_spy['CrashReturn'],
            'RecoveryDays': m_spy['RecoveryDays'],
            'StartDate': m_spy['StartDate'],
            'EndDate': m_spy['EndDate'],
            'TroughDate': m_spy['TroughDate']
        })

    # PLOT: normalized performance
    fig, ax = plt.subplots(figsize=(10,5))
    ax.plot(port_norm.index, port_norm.values, label='Balanced (60/30/10)', linewidth=2)
    if spy_norm is not None:
        ax.plot(spy_norm.index, spy_norm.values, label='S&P 500 (SPY)', linewidth=2)
    ax.set_title(f"{ev_name} ({s} → {e}) — Balanced vs S&P 500")
    ax.set_ylabel('Index (start = 100)')
    ax.legend()
    fname = FIG_DIR / f"{ev_name}_balanced_vs_spy.png"
    fig.savefig(fname, bbox_inches='tight')
    plt.close(fig)

    # PLOT: drawdowns
    fig2, ax2 = plt.subplots(figsize=(10,4))
    port_dd = port_norm / port_norm.cummax() - 1.0
    ax2.plot(port_dd.index, port_dd.values * 100, label='Balanced Drawdown (%)')
    if spy_norm is not None:
        spy_dd = spy_norm / spy_norm.cummax() - 1.0
        ax2.plot(spy_dd.index, spy_dd.values * 100, label='SPY Drawdown (%)')
    ax2.set_title(f"{ev_name} Drawdowns")
    ax2.set_ylabel('Drawdown (%)')
    ax2.legend()
    fname2 = FIG_DIR / f"{ev_name}_drawdown.png"
    fig2.savefig(fname2, bbox_inches='tight')
    plt.close(fig2)

    if VERBOSE:
        print(f"Saved charts for {ev_name}: {fname.name}, {fname2.name}")

# assemble metrics table
metrics_df = pd.DataFrame(rows)
metrics_df.to_csv(OUT_DIR / 'per_event_metrics.csv', index=False)
metrics_df.to_excel(OUT_DIR / 'per_event_metrics.xlsx', index=False)

print("Wrote metrics to outputs/per_event_metrics.* and saved figures to outputs/figs/")

# preview
if not metrics_df.empty:
    display(metrics_df.head(20))
else:
    print('No event metrics generated — check event windows and downloaded price ranges')


Balanced portfolio available from: 2011-01-28 to 2025-09-03
SPY available from: 1995-01-03 to 2025-09-03
Skipping Dotcom: insufficient Balanced data in 2000-03-01→2002-10-31 (have 0 days)
Skipping GFC: insufficient Balanced data in 2007-10-01→2009-03-31 (have 0 days)
Saved charts for COVID: COVID_balanced_vs_spy.png, COVID_drawdown.png
Saved charts for 2022_rates: 2022_rates_balanced_vs_spy.png, 2022_rates_drawdown.png
Skipping Asian_1997: insufficient Balanced data in 1997-07-01→1998-12-31 (have 0 days)
Saved charts for Debt_Ceiling_2011: Debt_Ceiling_2011_balanced_vs_spy.png, Debt_Ceiling_2011_drawdown.png
Wrote metrics to outputs/per_event_metrics.* and saved figures to outputs/figs/


,Event,Ticker,CAGR,Volatility,MaxDrawdown,CrashReturn,RecoveryDays,StartDate,EndDate,TroughDate
0,COVID,Balanced_60_30_10,-0.127832,0.289805,-0.201830,-0.201830,NaN,2020-02-01,2020-04-30,2020-03-23
1,COVID,SPY,-0.326501,0.495960,-0.337173,-0.337173,NaN,2020-02-01,2020-04-30,2020-03-23
2,2022_rates,Balanced_60_30_10,-0.205735,0.132167,-0.211152,-0.211152,NaN,2022-01-01,2022-10-31,2022-10-14
3,2022_rates,SPY,-0.209800,0.202735,-0.244964,-0.244964,NaN,2022-01-01,2022-10-31,2022-10-12
4,Debt_Ceiling_2011,Balanced_60_30_10,-0.009236,0.177120,-0.078136,-0.078136,24.0,2011-08-01,2011-12-30,2011-10-03
5,Debt_Ceiling_2011,SPY,-0.034039,0.320816,-0.141957,-0.141957,24.0,2011-08-01,2011-12-30,2011-10-03
